# Remote Speedback Automation

In [ ]:
import itertools
from datetime import time, timedelta, datetime

import the names of the people participating and their room link from a txt file:

In [ ]:
with open("speedback_participants.txt") as speedback_participants:
    list_of_participants = list(speedback_participants.readlines())
    speedback_participants = [line.rstrip('\n') for line in list_of_participants]

creating a dictionary from names to links to enable a lookup later

In [ ]:
participants_mapped_to_links = { participant.split(",")[0] : participant.split(", ")[1] for participant in speedback_participants }

Now we don't need the links for now. Let's get rid of them.

In [ ]:
participants = list(participants_mapped_to_links.keys())

Defining the first function that will give us the simple pairing possibilities.

In [ ]:
def build_pairings(list_of_pariticipants_names):
    create_pairings = list(itertools.combinations(participants, 2))
    return [list(pair) for pair in create_pairings]

In [ ]:
pairings = build_pairings(participants)

In [ ]:
def is_not_in_iteration(participant, iteration):
    hasnt_yet_given_feedback = True
    for pair in iteration:
        if participant in pair:
            hasnt_yet_given_feedback = False
    return hasnt_yet_given_feedback

def populate_grid(grid, pairs):
    index = 0
    if(index>=len(pairs)):
        return (True, grid)
    for row in range(len(grid)):
        print(row)
        for pair in pairs:
            print(pair)
            if(is_not_in_iteration(pair[0], grid[row]) and is_not_in_iteration(pair[1], grid[row])):
                for columnIndex in range(len(grid[row])):
                    if grid[row][columnIndex] == "None":
                        grid[row][columnIndex] = pair
                        index += 1
                        pairs.remove(pair)
                        break
    return (grid)

def containsCommonMember(self, other):
        return (self.__class__ == other.__class__ and
        (self.member1==other.member1 or
        self.member1==other.member2 or
        self.member2==other.member1 or
        self.member2==other.member2))

def not_in_iteration(self, participant, grid, rowIndex):
        result = False
        for element in grid[rowIndex]:
            if(pair.containsCommonMember(element)):
                result = False
                break
            if(element is None):
                result = True
        return result

grid[1][1]

populate_grid(grid, pairings)

Assign Pairs to rounds.

Utility Function:

Create rudementary matrix:

TODO: Rotationsprinzip = everyone is rotating im Uhrzeigersinn, except for one person

In [ ]:
def super_efficient_pairing_matrix(member):
    # non_mover = member[0]
    # if len(list)/2 % != 0 list.append("self-reflection")
    matrix = []
    this = int(len(member)/2)
    list_a = [i for i in member[:this]]
    list_b = [i for i in member[this:]]

    for index in range(len(list_a)):
        matrix.append([list_a[index], list_b[index]])

    for index in range(len(list_a)):
        b = list_a[:]
        b.insert((index+1), list_a[index])
        print(b)
    
    #print(list_a)
    #print(list_b)
    print(matrix)

In [ ]:
super_efficient_pairing_matrix(participants)

In [ ]:
def pairing_matrix(feedback_pairs):
    iteration = [["", ""]]
    matrix = []
    
    while len(feedback_pairs) > 0:
        for pair in feedback_pairs:
            if is_not_in_iteration(pair[0], iteration) & is_not_in_iteration(pair[1], iteration):
                iteration.append(pair)
                feedback_pairs.remove(pair)
        matrix.append(iteration[1:])
        iteration = [["", ""]]
        
    matrix[-2] = matrix[-2] + matrix[-1]
    matrix.pop()
    return matrix

In [ ]:
matrix = pairing_matrix(pairings)

Tasks:
   - assign time per round
   - assign room per pair
   - assure client have their room chosen
   - format for copy paste

In [ ]:
def assign_time(start_time, pairing_matrix):
    current_time = datetime.strptime(start_time, "%H")
    new_time = current_time + timedelta(minutes=8)
    #add_time_list = [iteration.insert(0, new_time) for iteration in len(pairing_matrix)]
    
    for iteration in pairing_matrix:
        new_time = current_time + timedelta(minutes=8)
        string_time = "{} - {}".format(current_time.time(), new_time.time())
        iteration.insert(0, string_time)
        current_time = new_time + timedelta(minutes=2)
    
    return pairing_matrix

In [ ]:
assign_time("10", matrix)

In [ ]:
def find_correct_link(pair, dict_participants_links):
    #if dict_participants_links.get(pair[0]).contains("client") or dict_participants_links.get(pair[1]).contains("client"):
    person_a = dict_participants_links.get(pair[0])
    person_b = dict_participants_links.get(pair[1])
    person_a_is_client = "client" in person_a
    person_b_is_client = "client" in person_b
    if person_a_is_client:
        return person_a.rsplit("client; ")[1]
    elif person_b_is_client:
        return person_b.rsplit("client; ")[1]
    elif "zoom" in person_a:
        return person_a
    elif person_b == "no_room":
        return person_a
    else:
        return person_b

In [ ]:
def assign_room(pairing_matrix, dict_participants_links):
    for iteration in pairing_matrix:
        for pair in iteration[1:]:
            link_to_append = find_correct_link(pair, dict_participants_links)
            pair.append(link_to_append)
    
    return pairing_matrix

In [ ]:
matrix = assign_room(matrix, participants_mapped_to_links)

In [ ]:
def formatting(ready_pairing_matrix):
    for iteration in ready_pairing_matrix:
        print(iteration[0], "\r\n")
        for item in iteration[1:]:
            print("{}, {} - {}".format(item[0], item[1], item[2]))
        print("\r\n")

In [ ]:
formatting(matrix)